## GRID3 User Survey Raw Aggregate Responses

In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_df = pd.read_excel("mel_survey_data_analysis.xlsx")

In [3]:
def clean_countries(df):
    import string
    for i in [0,15]:
        df.iloc[:,i] = df.iloc[:,i].apply(lambda x: string.capwords(x.lower()))
    #combining the same object encoded differently
    df.iloc[:,0] = df.iloc[:,0].replace("Guinée","Guinee Conakry")
    return df

In [4]:
main_col_names = [
    'In which country do you work?',
    'Do you use geospatial software and datasets (json, shp, kml, gpx, etc...) in your work?',
    'Which of the following best characterizes your relationship with GRID3?',
    'How did you become familiar with GRID3-supported products and work (geospatial data, analysis, maps, training)?',
    'Please specify "other"',
    'GRID3 enables stakeholders to access geospatial information that is of a higher quality (or otherwise improved), than what is currently available outside the program',
    'GRID3 fills gaps or complements existing mechanisms for geospatial data collection, creation, and production',
    'Have you used GRID3-supported products (geospatial data, maps, analyses) in your professional work?',
    'How likely are you to use GRID3-supported products (geospatial data, maps, analyses) in the future?',
    'Overall, what is your opinion of GRID3?',
    'Do you know anyone outside your organization who is using GRID3-supported products (geospatial data, maps, analyses) in their professional work?']

In [5]:
branch1_names = [
    'What do you think are the MOST useful areas of GRID3 support?',
    'What do you think are the LEAST useful areas of GRID3 support?',
]

branch2_names = [
    'Have you used GRID3-supported products (geospatial data, maps, analyses) in your professional work?'
    'When did you first start engaging with or using GRID3-supported products (geospatial data, analysis, maps)?'
    'Please briefly describe the project(s) for which you used GRID3-supported products',
    'Which GRID3-supported geospatial data were used?',
    '',
    '',
    'Have you used GRID3-supported maps or data hubs?',
    'Please select the products you or your organization have used',
    'Please rate the user-experience of GRID3-supported maps, relative to other maps or methods',
    'How would you characterize the user-experience (ease of use, design, relevance) of GRID3-supported hubs, relative to other data hubs?',
    'GRID3-supported products correctly identify the intended target LOCATIONS for my organization’s program or service',
    'GRID3-supported products correctly identify the intended target POPULATION for my organization’s program or service',
    'What, if any, barrier(s) prevent the use of GRID3-supported products (geospatial data, maps, analyses)?']

In [6]:
#cleaning countries
working_df = clean_countries(raw_df)

In [7]:
def search_col_idx(df,name):
    return df.columns.get_loc(name)

In [8]:
def get_name_idx_pair(df,lst):
    ret_lst = []
    for name in lst:
        idx = search_col_idx(df,name)
        ret_lst.append((name,idx))
    return ret_lst

In [9]:
main_cols = get_name_idx_pair(working_df,main_col_names)

In [10]:
def get_response_info_per_question(df,col_idx):
    pct_series = df.iloc[:,col_idx].value_counts(normalize=True)
    pct_series = pct_series.apply(lambda x: "{:.2%}".format(x))
    count = df.iloc[:,col_idx].count()
    return (pct_series.name,pct_series,count)

In [11]:
def get_all_responses(df,cols):
    ret_lst = []
    for name,idx in cols:
        info = get_response_info_per_question(df,idx)
        ret_lst.append((idx,*info))
    return ret_lst

In [12]:
main_col_info = get_all_responses(working_df,main_cols)

In [13]:
i = working_df.shape[1]
all_info = get_all_responses(working_df,zip([None]*i,list(range(i))))

In [14]:
pd.set_option('display.max_rows', None)

In [15]:
def clean_name(string):
    return string.replace('?','').replace('/','-').replace('*','')[:30]

In [18]:
all_series = [(x[2],x[3]) for x in all_info]
writer = pd.ExcelWriter('summaries.xlsx',engine='xlsxwriter')
c = 0
for ser,count in all_series:
    #print(count)
    #print(ser.name)
    df = pd.DataFrame(pd.concat([ser, pd.Series([count],index=['---N_COUNT_RESPONSES---']).rename('COUNT')]))
    df = df.rename(columns={0:ser.name})
    df.to_excel(writer,sheet_name=clean_name(ser.name))
    display(df)
    print()

writer.save()

,In which country do you work?
Nigeria,27.74%
Zambia,20.44%
Drc,19.71%
Burkina Faso,4.38%
Sierra Leone,3.65%
Mozambique,3.65%
Ghana,2.19%
Kenya,2.19%
South Sudan,2.19%
Somalia,1.46%


,"Do you use geospatial software and datasets (json, shp, kml, gpx, etc...) in your work?"
Yes,84.67%
No,15.33%
---N_COUNT_RESPONSES---,137


,Which of the following best characterizes your relationship with GRID3?
I (or my organization) am working or have previously worked with GRID3 on a project,69.34%
I (or my organization) have not worked with GRID3 on a project,30.66%
---N_COUNT_RESPONSES---,137


,"How did you become familiar with GRID3-supported products and work (geospatial data, analysis, maps, training)?"
Other,35.77%
GRID3 website,25.55%
GRID3 website Other,9.49%
GRID3 newsletter GRID3 website,4.38%
GRID3 website Whatsapp,2.19%
GRID3 newsletter,2.19%
GRID3 newsletter GRID3 website Other,2.19%
GRID3 newsletter GRID3 website Youtube,1.46%
GRID3 newsletter GRID3 website Whatsapp Other,1.46%
GRID3 website Youtube,1.46%


,"How did you become familiar with GRID3-supported products and work (geospatial data, analysis, maps, training)?/GRID3 newsletter"
0,80.29%
1,19.71%
---N_COUNT_RESPONSES---,137


...

,"Do you know anyone outside your organization who is using GRID3-supported products (geospatial data, maps, analyses) in their professional work?"
No,67.88%
Yes,32.12%
---N_COUNT_RESPONSES---,137


,_status
submitted_via_web,100.00%
---N_COUNT_RESPONSES---,137


,_submitted_by
---N_COUNT_RESPONSES---,0


,_tags
---N_COUNT_RESPONSES---,0


,_index
1,0.73%
95,0.73%
89,0.73%
90,0.73%
91,0.73%
92,0.73%
93,0.73%
94,0.73%
96,0.73%
70,0.73%
